In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('accepted_2007_to_2018Q4.csv.gz', engine='python', usecols=[2,5,6,8,9,11,12,13,15,16,20,22,23,24,25,26,27,28,29,30,31,32,33,53,54,57,58,60,69,110,115,116,117,118,119,120,121,122,123,124,125,126,127,34,35,36,61,62,63,64,65,66,67,68,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,111,112,113,114])
df.dropna(axis=0, how='all', inplace=True)
df['issue_d'] = pd.to_datetime(df['issue_d'], format='%b-%Y')

In [3]:
df['issue_d']

0         2015-12-01
1         2015-12-01
2         2015-12-01
3         2015-12-01
4         2015-12-01
             ...    
2260694   2016-10-01
2260695   2016-10-01
2260696   2016-10-01
2260697   2016-10-01
2260698   2016-10-01
Name: issue_d, Length: 2260668, dtype: datetime64[ns]

In [14]:
review = pd.DataFrame({'Unique Count':df.nunique(), 'dtype':df.dtypes})
#o_review = df[review[review['dtype'] != 'float64'].index.to_list()]
o_review = review[review['dtype'] != 'float64'].copy(deep=True)

o_review['unique values'] = [df[x].unique() for x in o_review.index]
''' 
for x in o_review.columns:
    print(o_review[x].unique())
    ''' 
o_review

Unique Count           dtype  \
term                                 2          object   
grade                                7          object   
sub_grade                           35          object   
emp_length                          11          object   
home_ownership                       6          object   
issue_d                            139  datetime64[ns]   
loan_status                          9          object   
purpose                             14          object   
zip_code                           956          object   
addr_state                          51          object   
earliest_cr_line                   754          object   
sec_app_earliest_cr_line           663          object   

                                                              unique values  
term                                               [ 36 months,  60 months]  
grade                                                 [C, B, F, A, E, D, G]  
sub_grade                 [C4, C1, B4, C5, F1, C3, B2, B1, A2, B5, C2, E...  
emp_length                [10+ years, 3 years, 4 years, 6 years, 1 year,...  
home_ownership                      [MORTGAGE, RENT, OWN, ANY, NONE, OTHER]  
issue_d                   [2015-12-01 00:00:00, 2015-11-01 00:00:00, 201...  
loan_status               [Fully Paid, Current, Charged Off, In Grace Pe...  
purpose                   [debt_consolidation, small_business, home_impr...  
zip_code                  [190xx, 577xx, 605xx, 076xx, 174xx, 300xx, 550...  
addr_state                [PA, SD, IL, NJ, GA, MN, SC, RI, TX, NC, CA, V...  
earliest_cr_line          [Aug-2003, Dec-1999, Aug-2000, Sep-2008, Jun-1...  
sec_app_earliest_cr_line  [nan, Feb-2005, Oct-1987, Feb-2013, Mar-2003, ...

In [6]:
df['emp_length'].unique()

array(['10+ years', '3 years', '4 years', '6 years', '1 year', '7 years',
       '8 years', '5 years', '2 years', '9 years', '< 1 year', nan],
      dtype=object)

In [8]:
df.groupby([pd.Grouper(key='issue_d', freq='YE'),'home_ownership']).size()

issue_d     home_ownership
2007-12-31  MORTGAGE             206
            NONE                   7
            OWN                   52
            RENT                 338
2008-12-31  MORTGAGE             931
            NONE                   1
            OTHER                 47
            OWN                  148
            RENT                1266
2009-12-31  MORTGAGE            2048
            OTHER                 85
            OWN                  472
            RENT                2676
2010-12-31  MORTGAGE            5617
            OTHER                  3
            OWN                  963
            RENT                5954
2011-12-31  MORTGAGE           10157
            OTHER                  1
            OWN                 1616
            RENT                9947
2012-12-31  MORTGAGE           24947
            NONE                  42
            OTHER                 46
            OWN                 4203
            RENT               24129
2013-12-31 

In [3]:
df.replace(['ANY', 'NONE', 'OTHER'], 'INDETERMINATE', inplace=True)

In [5]:
pd.get_dummies(df['term'], prefix='term', dtype=int)


term_ 36 months  term_ 60 months
0                      1                0
1                      1                0
2                      0                1
3                      0                1
4                      0                1
...                  ...              ...
2260694                0                1
2260695                0                1
2260696                0                1
2260697                0                1
2260698                0                1

[2260668 rows x 2 columns]

In [6]:
df[df['zip_code'].isnull()]['addr_state'] #drop this value as it is only one row of troublesome data


Series([], Name: addr_state, dtype: object)

In [5]:
df.drop(1736050, axis=0, inplace=True)

In [8]:
df['addr_state'] + df.zip_code.str[:-2]

0          PA190
1          SD577
2          IL605
3          NJ076
4          PA174
           ...  
2260694    AL356
2260695    UT840
2260696    CA907
2260697    FL334
2260698    TX770
Length: 2260667, dtype: object

In [9]:
state_abbrev_fips_dict = { #these have been converted from the standard pattern of padded two digit int to allow the addition of 3 digit zip later.
 'AL': 1000,
 'AK': 2000,
 'AZ': 4000,
 'AR': 5000,
 'CA': 6000,
 'CO': 8000,
 'CT': 9000,
 'DE': 10000,
 'FL': 12000,
 'GA': 13000,
 'HI': 15000,
 'ID': 16000,
 'IL': 17000,
 'IN': 18000,
 'IA': 19000,
 'KS': 20000,
 'KY': 21000,
 'LA': 22000,
 'ME': 23000,
 'MD': 24000,
 'MA': 25000,
 'MI': 26000,
 'MN': 27000,
 'MS': 28000,
 'MO': 29000,
 'MT': 30000,
 'NE': 31000,
 'NV': 32000,
 'NH': 33000,
 'NJ': 34000,
 'NM': 35000,
 'NY': 36000,
 'NC': 37000,
 'ND': 38000,
 'OH': 39000,
 'OK': 40000,
 'OR': 41000,
 'PA': 42000,
 'RI': 44000,
 'SC': 45000,
 'SD': 46000,
 'TN': 47000,
 'TX': 48000,
 'UT': 49000,
 'VT': 50000,
 'VA': 51000,
 'WA': 53000,
 'WV': 54000,
 'WI': 55000,
 'WY': 56000,
 'AS': 60000,
 'GU': 66000,
 'MP': 69000,
 'PR': 72000,
 'VI': 78000}

In [12]:
df['addr_state'].map(state_abbrev_fips_dict) + df.zip_code.str[:-2].astype(int)

0          42190.0
1          46577.0
2          17605.0
3          34076.0
4          42174.0
            ...   
2260694     1356.0
2260695    49840.0
2260696     6907.0
2260697    12334.0
2260698    48770.0
Length: 2260667, dtype: float64

In [ ]:
term_du = pd.get_dummies(df['term'], prefix='term', dtype=int)
grade = 	 pd.get_dummies(df['term'], prefix='term', dtype=int)

sub_grade	
emp_length	11	object	[10+ years, 3 years, 4 years, 6 years, 1 year,...
home_ownership	6	object	[MORTGAGE, RENT, OWN, ANY, NONE, OTHER]
loan_status	9	object	[Fully Paid, Current, Charged Off, In Grace Pe...
purpose	14	object	[debt_consolidation, small_business, home_impr...
zip_code	956	object	[190xx, 577xx, 605xx, 076xx, 174xx, 300xx, 550...
addr_state	51	object	[PA, SD, IL, NJ, GA, MN, SC, RI, TX, NC, CA, V...
earliest_cr_line	754	object	[Aug-2003, Dec-1999, Aug-2000, Sep-2008, Jun-1...
sec_app_earliest_cr_line

In [ ]:
'''
-binary varables
-is this useful
- is this useful?
-ranked ?
-group ANY, NONE, AND OTHER AS indeterminant.
-consolidate categories
-thurn into multiple binary varables?
-combine with state
-combine with zip
- caclualte as months before app date?
-cacluate as months before application date?

In [9]:
df.shape

(2260668, 97)

In [ ]:
ddf=da.from_pandas(df, chunksize=566000)
save_dir = 'data/'
ddf.to_parquet(save_dir)

In [12]:
df.loan_status.unique()

array(['Fully Paid', 'Current', 'Charged Off', 'In Grace Period',
       'Late (31-120 days)', 'Late (16-30 days)', 'Default',
       'Does not meet the credit policy. Status:Fully Paid',
       'Does not meet the credit policy. Status:Charged Off'],
      dtype=object)

In [10]:
df.tail()

loan_amnt        term  int_rate grade sub_grade emp_length  \
2260694    24000.0   60 months     12.79     C        C1    7 years   
2260695    24000.0   60 months     10.49     B        B3  10+ years   
2260696    40000.0   60 months     10.49     B        B3    9 years   
2260697    24000.0   60 months     14.49     C        C4    6 years   
2260698    14000.0   60 months     14.49     C        C4  10+ years   

        home_ownership  annual_inc  loan_status             purpose  ...  \
2260694       MORTGAGE     95000.0      Current    home_improvement  ...   
2260695       MORTGAGE    108000.0      Current  debt_consolidation  ...   
2260696       MORTGAGE    227000.0      Current  debt_consolidation  ...   
2260697           RENT    110000.0  Charged Off  debt_consolidation  ...   
2260698       MORTGAGE     95000.0      Current  debt_consolidation  ...   

        sec_app_earliest_cr_line sec_app_inq_last_6mths  sec_app_mort_acc  \
2260694                      NaN                    NaN               NaN   
2260695                      NaN                    NaN               NaN   
2260696                      NaN                    NaN               NaN   
2260697                      NaN                    NaN               NaN   
2260698                      NaN                    NaN               NaN   

         sec_app_open_acc sec_app_revol_util  sec_app_open_act_il  \
2260694               NaN                NaN                  NaN   
2260695               NaN                NaN                  NaN   
2260696               NaN                NaN                  NaN   
2260697               NaN                NaN                  NaN   
2260698               NaN                NaN                  NaN   

         sec_app_num_rev_accts  sec_app_chargeoff_within_12_mths  \
2260694                    NaN                               NaN   
2260695                    NaN                               NaN   
2260696                    NaN                               NaN   
2260697                    NaN                               NaN   
2260698                    NaN                               NaN   

         sec_app_collections_12_mths_ex_med  \
2260694                                 NaN   
2260695                                 NaN   
2260696                                 NaN   
2260697                                 NaN   
2260698                                 NaN   

         sec_app_mths_since_last_major_derog  
2260694                                  NaN  
2260695                                  NaN  
2260696                                  NaN  
2260697                                  NaN  
2260698                                  NaN  

[5 rows x 97 columns]

#### Data Values in this set are as follows:

- id: A unique LC assigned ID for the loan listing.
- member_id: A unique LC assigned Id for the borrower member.
- loan_amnt: The listed amount of the loan applied for by the borrower. If at some point in time, the credit department reduces the loan amount, then it will be reflected in this value.
- funded_amnt: The total amount committed to that loan at that point in time.
- funded_amnt_inv: The total amount committed by investors for that loan at that point in time.
- term: The number of payments on the loan. Values are in months and can be either 36 or 60.
- int_rate: Interest Rate on the loan
- installment: The monthly payment owed by the borrower if the loan originates.
- grade: LC assigned loan grade
- sub_grade: LC assigned loan subgrade
- emp_title: The job title supplied by the Borrower when applying for the loan.*
- emp_length: Employment length in years. Possible values are between 0 and 10 where 0 means less than one year and 10 means ten or more years. 
- home_ownership: The home ownership status provided by the borrower during registration or obtained from the credit report. Our values are: RENT, OWN, MORTGAGE, OTHER
- annual_inc: The self-reported annual income provided by the borrower during registration.
- verification_status: Indicates if income was verified by LC, not verified, or if the income source was verified
- issue_d: The month which the loan was funded
- loan_status: Current status of the loan
- pymnt_plan: Indicates if a payment plan has been put in place for the loan
- url: URL for the LC page with listing data.
- desc: Loan description provided by the borrower
- purpose: A category provided by the borrower for the loan request. 
- title: The loan title provided by the borrower
- zip_code: The first 3 numbers of the zip code provided by the borrower in the loan application.
- addr_state: The state provided by the borrower in the loan application
- dti: A ratio calculated using the borrower’s total monthly debt payments on the total debt obligations, excluding mortgage and the requested LC loan, divided by the borrower’s self-reported monthly income.
- delinq_2yrs: The number of 30+ days past-due incidences of delinquency in the borrower's credit file for the past 2 years
- earliest_cr_line: The month the borrower's earliest reported credit line was opened
- fico_range_low: The lower boundary range the borrower’s FICO at loan origination belongs to.
- fico_range_high: The upper boundary range the borrower’s FICO at loan origination belongs to.
- inq_last_6mths: The number of inquiries in past 6 months (excluding auto and mortgage inquiries)
- mths_since_last_delinq: The number of months since the borrower's last delinquency.
- mths_since_last_record: The number of months since the last public record.
- open_acc: The number of open credit lines in the borrower's credit file.
- pub_rec: Number of derogatory public records
- revol_bal: Total credit revolving balance
- revol_util: Revolving line utilization rate, or the amount of credit the borrower is using relative to all available revolving credit.
- total_acc: The total number of credit lines currently in the borrower's credit file
- initial_list_status: The initial listing status of the loan. Possible values are – W, F
- out_prncp: Remaining outstanding principal for total amount funded
- out_prncp_inv: Remaining outstanding principal for portion of total amount funded by investors
- total_pymnt: Payments received to date for total amount funded
- total_pymnt_inv: Payments received to date for portion of total amount funded by investors
- total_rec_prncp: Principal received to date
- total_rec_int: Interest received to date
- total_rec_late_fee: Late fees received to date
- recoveries: post charge off gross recovery
- collection_recovery_fee: post charge off collection fee
- last_pymnt_d: Last month payment was received
- last_pymnt_amnt: Last total payment amount received
- next_pymnt_d: Next scheduled payment date
- last_credit_pull_d: The most recent month LC pulled credit for this loan
- last_fico_range_high: The upper boundary range the borrower’s last FICO pulled belongs to.
- last_fico_range_low: The lower boundary range the borrower’s last FICO pulled belongs to.
- collections_12_mths_ex_med: Number of collections in 12 months excluding medical collections
- mths_since_last_major_derog: Months since most recent 90-day or worse rating
- policy_code: publicly available policy_code=1 new products not publicly available policy_code=2
- application_type: Indicates whether the loan is an individual application or a joint application with two co-borrowers
- annual_inc_joint: The combined self-reported annual income provided by the co-borrowers during registration
- dti_joint: A ratio calculated using the co-borrowers' total monthly payments on the total debt obligations, excluding mortgages and the requested LC loan, divided by the co-borrowers' combined self-reported monthly income
- verification_status_joint: Indicates if the co-borrowers' joint income was verified by LC, not verified, or if the income source was verified
- acc_now_delinq: The number of accounts on which the borrower is now delinquent.
- tot_coll_amt: Total collection amounts ever owed
- tot_cur_bal: Total current balance of all accounts
- open_acc_6m: Number of open trades in last 6 months
- open_act_il: Number of currently active installment trades
- open_il_12m: Number of installment accounts opened in past 12 months
- open_il_24m: Number of installment accounts opened in past 24 months
- mths_since_rcnt_il: Months since most recent installment accounts opened
- total_bal_il: Total current balance of all installment accounts
- il_util: Ratio of total current balance to high credit/credit limit on all install acct
- open_rv_12m: Number of revolving trades opened in past 12 months
- open_rv_24m: Number of revolving trades opened in past 24 months
- max_bal_bc: Maximum current balance owed on all revolving accounts
- all_util: Balance to credit limit on all trades
- total_rev_hi_lim: Total revolving high credit/credit limit
- inq_fi: Number of personal finance inquiries
- total_cu_tl: Number of finance trades
- inq_last_12m: Number of credit inquiries in past 12 months
- acc_open_past_24mths: Number of trades opened in past 24 months.
- avg_cur_bal: Average current balance of all accounts
- bc_open_to_buy: Total open to buy on revolving bankcards.
- bc_util: Ratio of total current balance to high credit/credit limit for all bankcard accounts.
- chargeoff_within_12_mths: Number of charge-offs within 12 months
- delinq_amnt: The past-due amount owed for the accounts on which the borrower is now delinquent.
- mo_sin_old_il_acct: Months since oldest bank installment account opened
- mo_sin_old_rev_tl_op: Months since oldest revolving account opened
- mo_sin_rcnt_rev_tl_op: Months since most recent revolving account opened
- mo_sin_rcnt_tl: Months since most recent account opened
- mort_acc: Number of mortgage accounts.
- mths_since_recent_bc: Months since most recent bankcard account opened.
- mths_since_recent_bc_dlq: Months since most recent bankcard delinquency
- mths_since_recent_inq: Months since most recent inquiry.
- mths_since_recent_revol_delinq: Months since most recent revolving delinquency.
- num_accts_ever_120_pd: Number of accounts ever 120 or more days past due
- num_actv_bc_tl: Number of currently active bankcard accounts
- num_actv_rev_tl: Number of currently active revolving trades
- num_bc_sats: Number of satisfactory bankcard accounts
- num_bc_tl: Number of bankcard accounts
- num_il_tl: Number of installment accounts
- num_op_rev_tl: Number of open revolving accounts
- num_rev_accts: Number of revolving accounts
- num_rev_tl_bal_gt_0: Number of revolving trades with balance >0
- num_sats: Number of satisfactory accounts
- num_tl_120dpd_2m: Number of accounts currently 120 days past due (updated in past 2 months)
- num_tl_30dpd: Number of accounts currently 30 days past due (updated in past 2 months)
- num_tl_90g_dpd_24m: Number of accounts 90 or more days past due in last 24 months
- num_tl_op_past_12m: Number of accounts opened in past 12 months
- pct_tl_nvr_dlq: Percent of trades never delinquent
- percent_bc_gt_75: Percentage of all bankcard accounts > 75% of limit.
- pub_rec_bankruptcies: Number of public record bankruptcies
- tax_liens: Number of tax liens
- tot_hi_cred_lim: Total high credit/credit limit
- total_bal_ex_mort: Total credit balance excluding mortgage
- total_bc_limit: Total bankcard high credit/credit limit
- total_il_high_credit_limit: Total installment high credit/credit limit
- revol_bal_joint:  Sum of revolving credit balance of the co-borrowers, net of duplicate balances
- sec_app_fico_range_low:  FICO range (high) for the secondary applicant
- sec_app_fico_range_high:  FICO range (low) for the secondary applicant
- sec_app_earliest_cr_line:  Earliest credit line at time of application for the secondary applicant
- sec_app_inq_last_6mths:  Credit inquiries in the last 6 months at time of application for the secondary applicant
- sec_app_mort_acc:  Number of mortgage accounts at time of application for the secondary applicant
- sec_app_open_acc:  Number of open trades at time of application for the secondary applicant
- sec_app_revol_util:  Ratio of total current balance to high credit/credit limit for all revolving accounts
- sec_app_open_act_il:  Number of currently active installment trades at time of application for the secondary applicant
- sec_app_num_rev_accts:  Number of revolving accounts at time of application for the secondary applicant
- sec_app_chargeoff_within_12_mths:  Number of charge-offs within last 12 months at time of application for the secondary applicant
- sec_app_collections_12_mths_ex_med:  Number of collections within last 12 months excluding medical collections at time of application for the secondary applicant
- sec_app_mths_since_last_major_derog:  Months since most recent 90-day or worse rating at time of application for the secondary applicant
- hardship_flag: Flags whether or not the borrower is on a hardship plan
- hardship_type: Describes the hardship plan offering
- hardship_reason: Describes the reason the hardship plan was offered
- hardship_status: Describes if the hardship plan is active, pending, canceled, completed, or broken
- deferral_term: Amount of months that the borrower is expected to pay less than the contractual monthly payment amount due to a hardship plan
- hardship_amount: The interest payment that the borrower has committed to make each month while they are on a hardship plan
- hardship_start_date: The start date of the hardship plan period
- hardship_end_date: The end date of the hardship plan period
- payment_plan_start_date: The day the first hardship plan payment is due. For example, if a borrower has a hardship plan period of 3 months, the start date is the start of the three-month period in which the borrower is allowed to make interest-only payments.
- hardship_length: The number of months the borrower will make smaller payments than normally obligated due to a hardship plan
- hardship_dpd: Account days past due as of the hardship plan start date
- hardship_loan_status: Loan Status as of the hardship plan start date
- orig_projected_additional_accrued_interest: The original projected additional interest amount that will accrue for the given hardship payment plan as of the Hardship Start Date. This field will be null if the borrower has broken their hardship payment plan.
- hardship_payoff_balance_amount: The payoff balance amount as of the hardship plan start date
- hardship_last_payment_amount: The last payment amount as of the hardship plan start date
- disbursement_method: The method by which the borrower receives their loan. Possible values are: CASH, DIRECT_PAY
- debt_settlement_flag: Flags whether or not the borrower, who has charged-off, is working with a debt-settlement company.
- debt_settlement_flag_date: The most recent date that the Debt_Settlement_Flag has been set  
- settlement_status: The status of the borrower’s settlement plan. Possible values are: COMPLETE, ACTIVE, BROKEN, CANCELLED, DENIED, DRAFT
- settlement_date: The date that the borrower agrees to the settlement plan
- settlement_amount: The loan amount that the borrower has agreed to settle for
- settlement_percentage: The settlement amount as a percentage of the payoff balance amount on the loan
- settlement_term: The number of months that the borrower will be on the settlement plan


\* Employer Title replaces Employer Name for all loans listed after 9/23/2013

In [4]:
import pprint
pprint.pp(df.columns.values.tolist())

['id',
 'member_id',
 'loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'term',
 'int_rate',
 'installment',
 'grade',
 'sub_grade',
 'emp_title',
 'emp_length',
 'home_ownership',
 'annual_inc',
 'verification_status',
 'issue_d',
 'loan_status',
 'pymnt_plan',
 'url',
 'desc',
 'purpose',
 'title',
 'zip_code',
 'addr_state',
 'dti',
 'delinq_2yrs',
 'earliest_cr_line',
 'fico_range_low',
 'fico_range_high',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'initial_list_status',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_d',
 'last_pymnt_amnt',
 'next_pymnt_d',
 'last_credit_pull_d',
 'last_fico_range_high',
 'last_fico_range_low',
 'collections_12_mths_ex_med',
 'mths_since_last_major_derog',
 'policy_code',
 'application_type',
 'annual_inc_joint',
 '

FICO not useful for Basil framework as that is based on individual american consumers only and not reflective of overall bank health. 

In [15]:
df.to_parquet('test.parquet.gzip', compression='gzip')